<a href="https://colab.research.google.com/github/cs449w23/project-cs_get_degrees/blob/main/CNN_rodney.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required 3rd party packages 
%pip install h5py

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Import required libs 
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from collections import defaultdict
from torch.utils.data import Dataset, random_split, TensorDataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.image as mpimg
from torch.utils.data import DataLoader
import time
import h5py

In [ ]:
# Define CNN class
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        #self.linear_layer_size = linear_layer_size
        self.conv1 = torch.nn.Conv2d(3, 16, kernel_size=3)  # input: torch.Size([64, 3, 96, 96])
        self.conv2 = torch.nn.Conv2d(16, 32, kernel_size=3) # input: torch.Size([64, 16, 47, 47])
        self.conv3 = torch.nn.Conv2d(32, 64, kernel_size=3) # input: torch.Size([64, 32, 22, 22])
        self.conv4 = torch.nn.Conv2d(64, 128, kernel_size=3) # input: torch.Size([64, 64, 10, 10])
        # output: torch.Size([64, 128, 4, 4])
        

    def forward(self, x):
        #print(x.shape)
        batch_size = x.size(0)

        relu = torch.nn.ReLU()
        maxpool = torch.nn.MaxPool2d(stride=2,kernel_size=2)
        x = self.conv1(x)
        x = maxpool(relu(x))
        #print(x.shape)
        x = self.conv2(x)
        x = maxpool(relu(x))
        #print(x.shape)
        x = self.conv3(x)
        x = maxpool(relu(x))
        #print(x.shape)
        x = self.conv4(x)
        x = maxpool(relu(x))
        #print(x.shape)
        # x = x.reshape(batch_size, self.linear_layer_size)
        # x = self.linear(x)
        return x

In [ ]:
# define Fully Connected Class
class FC(torch.nn.Module):
    def __init__(self):
          super(FC, self).__init__()
          self.fc1 = torch.nn.Linear(4 * 4 * 128, 1)  # input: (6,6,128) # output 2 classes
          self.actv = torch.nn.Sigmoid()
    def forward(self, x):
        #print(x.shape)
        batch_size = x.size(0)
        x = torch.flatten(x, 1)
        #print(x.shape)
        x = self.fc1(x)
        x = self.actv(x)
        return x


In [ ]:
# Define Recurrent Neural Network Class
class RNN(torch.nn.Module):
    def __init__(self, batch_size=32):
        super(RNN, self).__init__()
        self.input_size = 128
        self.output_size = 1
        self.hidden_size = 128
        self.n_layers = 1
        self.batch_size = batch_size
        # batch_first means that the first dim of the input and output will be the batch_size
        # self.rnn = torch.nn.RNN(self.input_size, self.hidden_size, self.n_layers, batch_first=True)
        # last, fully-connected layer
        self.rnn = torch.nn.RNNCell(self.input_size, self.hidden_size)
        self.fc = torch.nn.Linear(self.hidden_size*16, self.output_size)
        self.actv = torch.nn.Sigmoid()

    def forward(self, x):
        hidden = None
        batch_size = x.size(0)
        x = x.permute(0,2,3,1).reshape(batch_size,16,-1)
        output= torch.tensor([]).to(device)
        for pixel in range(16):
          input = x[:,pixel,:]
          hidden = self.rnn(input, hidden)
          # print("hidden: ", hidden.shape)
          output = torch.cat((output, hidden), dim=1)
        # get RNN outputs
        
        # shape output to be (batch_size*seq_length, hidden_dim)
        #r_out = r_out.view(-1, self.hidden_size)
        
        # get final output 
        output = self.fc(output)
        
        #return output, hidden
        return self.actv(output)
        
    # def initHidden(self):
    #     return torch.zeros(self.n_layers,self.batch_size, self.hidden_size)

In [ ]:
# Define Long Short-Term Memory Class
class LSTM(torch.nn.Module):
    def __init__(self,batch_size=32):
        super(LSTM,self).__init__()
        self.input_size = 128
        self.output_size = 1
        self.hidden_size = 128
        self.batch_size = batch_size
        self.lstm = torch.nn.LSTMCell(input_size=self.input_size,hidden_size=self.hidden_size)
        self.fc = torch.nn.Linear(in_features=self.hidden_size * 16,out_features=self.output_size)
        self.actv = torch.nn.Sigmoid()

    def forward(self,x):
        batch_size = x.size(0)
        hidden = torch.zeros(batch_size, self.hidden_size).to(device)
        cell_state = torch.zeros(batch_size, self.hidden_size).to(device)
        x = x.permute(0,2,3,1).reshape(batch_size,16,-1)
        output= torch.tensor([]).to(device)
        for pixel in range(16):
          input = x[:,pixel,:]
          (hidden, cell_state) = self.lstm(input, (hidden, cell_state))
          # print("hidden: ", hidden.shape)
          output = torch.cat((output, hidden), dim=1)
        # get RNN outputs
        
        # shape output to be (batch_size*seq_length, hidden_dim)
        #r_out = r_out.view(-1, self.hidden_size)
        
        # get final output 
        output = self.fc(output)
        
        #return output, hidden
        return self.actv(output)

    def initHidden(self):
        return torch.zeros(self.n_layers,self.batch_size, self.hidden_size)


In [ ]:
# Define Meta Data
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
display(device)
batch_size = 32
train_x_path = "/kaggle/input/metastatic-tissue-classification-patchcamelyon/pcam/training_split.h5"
train_y_path = "/kaggle/input/metastatic-tissue-classification-patchcamelyon/Labels/Labels/camelyonpatch_level_2_split_train_y.h5"
val_x_path = "/kaggle/input/metastatic-tissue-classification-patchcamelyon/pcam/validation_split.h5"
val_y_path = "/kaggle/input/metastatic-tissue-classification-patchcamelyon/Labels/Labels/camelyonpatch_level_2_split_valid_y.h5"
save_model_dir = "/kaggle/working/"

device(type='cuda', index=0)

In [ ]:
# Init Training Data
split_test = 75_000
with h5py.File(train_x_path, "r") as f:
    train_data_x = f['x']
    train_data_x_small = torch.from_numpy(train_data_x[:split_test]).permute(0,3,1,2).float()
    print(train_data_x_small.shape)
with h5py.File(train_y_path, "r") as f:
    train_data_y = f['y']
    train_data_y_small = torch.from_numpy(train_data_y[:split_test]).float()


train_data_small = TensorDataset(train_data_x_small, train_data_y_small)
train_dataloader = DataLoader(train_data_small, batch_size=batch_size, shuffle=True, pin_memory=True)

torch.Size([75000, 3, 96, 96])


In [ ]:
# Init Validation Data
split_val = 7_500
with h5py.File(val_x_path, "r") as f:
    val_data_x = f['x']
    val_data_x_small = torch.from_numpy(val_data_x[:split_val]).float().permute(0,3,1,2).float()
    print(val_data_x_small.shape)
with h5py.File(val_y_path, "r") as f:
    val_data_y = f['y']
    val_data_y_small = torch.from_numpy(val_data_y[:split_val]).float()
    
    
val_data_small = TensorDataset(val_data_x_small, val_data_y_small)
val_dataloader = DataLoader(val_data_small, batch_size=batch_size, shuffle=True, pin_memory=True)

torch.Size([7500, 3, 96, 96])


In [ ]:
# Define helper function to track length of epoch
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs
    

In [ ]:
# Run one epoch 
def run_one_epoch(epoch_idx, cnn, model_out, optimizer, train_dataloader, val_dataloader):
    start_time = time.time()
    cnn.train()
    model_out.train()
    # loop through the entire dataset once per epoch
    train_loss = 0.0
    train_acc = 0.0
    train_total = 0
    for batch_idx, (images, labels) in enumerate(train_dataloader):
        optimizer.zero_grad()
        images = images.to(device)
        labels = labels.to(device)
        cnn_output = cnn(images)
        #print("hi")
        predictions = model_out(cnn_output).squeeze()
        acc_sum = 0
        for i, pred in enumerate(predictions):
          if round(pred.item()) == labels[i]:
            acc_sum += 1
        acc = acc_sum / labels.size(0)
        # acc = torch.sum(predictions == labels) / labels.size(0)
#         print(predictions.shape, labels.squeeze().shape)
        loss = torch.nn.BCELoss()(predictions, labels.squeeze().float())
        loss.backward()
        optimizer.step()
    # update statistics
        train_loss += (loss * len(images))
        train_acc += (acc * len(labels))
        train_total += len(labels)
    
    train_loss /= train_total
    train_acc /= train_total
    cnn.eval()
    model_out.eval()
    val_loss = 0
    val_acc = 0
    val_total = 0
    with torch.no_grad():
      for batch_idx, (images, labels) in enumerate(val_dataloader):
          images = images.to(device)
          labels = labels.to(device)
          cnn_output = cnn(images)
          predictions = model_out(cnn_output).squeeze()
          acc_sum = 0
          for i, pred in enumerate(predictions):
            if round(pred.item()) == labels[i]:
              acc_sum += 1
          acc = acc_sum / labels.size(0)
          # acc = torch.sum(predictions == labels) / labels.size(0)
          loss = torch.nn.BCELoss()(predictions, labels.squeeze().float())
          # update statistics
          val_loss += (loss * len(images))
          val_acc += (acc * len(labels))
          val_total += len(labels)
      val_loss /= val_total
      val_acc /= val_total
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    print(f'Epoch: {epoch_idx+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f"Epoch {epoch_idx + 1}:  val loss {val_loss :0.3f}, val acc {val_acc :0.3f}, train loss {train_loss :0.3f}, train acc {train_acc :0.3f}")
    # Detach tells torch to stop tracking a tensor's gradients
    return train_acc, train_loss.detach(), val_acc,val_loss.detach()

In [ ]:
# Train the inputted Models
def train(cnn, model_out, epochs, lr, cnn_name="model_cnn", model_out_name="model_out"):
    train_accs = []
    train_losses = []
    val_accs = []
    val_losses = []
    best_val_loss = float('inf')
    opt = torch.optim.Adam([{"params": cnn.parameters()},{"params":model_out.parameters()}], lr=lr)
    for epoch in range(epochs):
      train_acc, train_loss, val_acc, val_loss = run_one_epoch(epoch, cnn, model_out, opt, train_dataloader, val_dataloader)
      train_accs.append(train_acc)
      train_losses.append(train_loss)
      val_accs.append(val_acc)
      val_losses.append(val_loss)
      if best_val_loss > val_loss:
            torch.save(cnn, save_model_dir + cnn_name)
            torch.save(model_out, save_model_dir + model_out_name)
            best_val_loss = val_loss
            print("Saved Model")
    return train_accs, train_losses, val_accs, val_losses

In [ ]:
# Init and train the fully connected model
cnn_fc = CNN()
cnn_fc.to(device)
fc = FC()
fc.to(device)
epochs = 10
lr = 1e-4

fc_train_accs, fc_train_losses, fc_val_accs, fc_val_losses = train(cnn_fc, 
                                                                   fc, 
                                                                   epochs, 
                                                                   lr, 
                                                                   cnn_name="cnn_fc", 
                                                                   model_out_name="model_out_fc")

Epoch: 01 | Time: 0m 17s
Epoch 1:  val loss 0.452, val acc 0.781, train loss 0.497, train acc 0.764
Epoch: 02 | Time: 0m 17s
Epoch 2:  val loss 0.461, val acc 0.777, train loss 0.435, train acc 0.802
Epoch: 03 | Time: 0m 17s
Epoch 3:  val loss 0.418, val acc 0.809, train loss 0.397, train acc 0.824
Epoch: 04 | Time: 0m 17s
Epoch 4:  val loss 0.506, val acc 0.755, train loss 0.367, train acc 0.840
Epoch: 05 | Time: 0m 17s
Epoch 5:  val loss 0.426, val acc 0.814, train loss 0.333, train acc 0.858
Epoch: 06 | Time: 0m 18s
Epoch 6:  val loss 0.459, val acc 0.801, train loss 0.304, train acc 0.872
Epoch: 07 | Time: 0m 17s
Epoch 7:  val loss 0.483, val acc 0.801, train loss 0.276, train acc 0.887
Epoch: 08 | Time: 0m 18s
Epoch 8:  val loss 0.444, val acc 0.811, train loss 0.248, train acc 0.900
Epoch: 09 | Time: 0m 17s
Epoch 9:  val loss 0.506, val acc 0.784, train loss 0.222, train acc 0.912
Epoch: 10 | Time: 0m 17s
Epoch 10:  val loss 0.697, val acc 0.738, train loss 0.196, train acc 0.923

In [ ]:
# init and train the RNN model
cnn_rrn = CNN()
cnn_rrn.to(device)
rrn = RNN()
rrn.to(device)
epochs = 10
lr = 1e-4

rnn_train_accs, rnn_train_losses, rnn_val_accs, rnn_val_losses = train(cnn_rrn, 
                                                                       rrn, 
                                                                       epochs, 
                                                                       lr, 
                                                                       cnn_name="cnn_rnn", 
                                                                       model_out_name="model_out_rnn")

Epoch: 01 | Time: 0m 33s
Epoch 1:  val loss 0.475, val acc 0.761, train loss 0.480, train acc 0.776
Saved Model
Epoch: 02 | Time: 0m 33s
Epoch 2:  val loss 0.440, val acc 0.782, train loss 0.428, train acc 0.806
Saved Model
Epoch: 03 | Time: 0m 33s
Epoch 3:  val loss 0.419, val acc 0.809, train loss 0.387, train acc 0.830
Saved Model
Epoch: 04 | Time: 0m 33s
Epoch 4:  val loss 0.396, val acc 0.822, train loss 0.346, train acc 0.853
Saved Model
Epoch: 05 | Time: 0m 32s
Epoch 5:  val loss 0.427, val acc 0.808, train loss 0.309, train acc 0.871
Epoch: 06 | Time: 0m 33s
Epoch 6:  val loss 0.440, val acc 0.810, train loss 0.282, train acc 0.883
Epoch: 07 | Time: 0m 33s
Epoch 7:  val loss 0.424, val acc 0.797, train loss 0.251, train acc 0.899
Epoch: 08 | Time: 0m 33s
Epoch 8:  val loss 0.481, val acc 0.804, train loss 0.228, train acc 0.908
Epoch: 09 | Time: 0m 33s
Epoch 9:  val loss 0.445, val acc 0.813, train loss 0.204, train acc 0.919
Epoch: 10 | Time: 0m 32s
Epoch 10:  val loss 0.637, 

In [ ]:
# init and train the LSTM model
cnn_lstm = CNN()
cnn_lstm.to(device)
lstm = LSTM()
lstm.to(device)
epochs = 10
lr = 1e-4

lstm_train_accs, lstm_train_losses, lstm_val_accs, lstm_val_losses = train(cnn_lstm, 
                                                                           lstm, 
                                                                           epochs, 
                                                                           lr, 
                                                                           cnn_name="cnn_lstm", 
                                                                           model_out_name="model_out_lstm")

Epoch: 01 | Time: 0m 33s
Epoch 1:  val loss 0.433, val acc 0.795, train loss 0.470, train acc 0.782
Saved Model
Epoch: 02 | Time: 0m 33s
Epoch 2:  val loss 0.497, val acc 0.764, train loss 0.430, train acc 0.807
Epoch: 03 | Time: 0m 33s
Epoch 3:  val loss 0.469, val acc 0.791, train loss 0.388, train acc 0.828
Epoch: 04 | Time: 0m 33s
Epoch 4:  val loss 0.373, val acc 0.827, train loss 0.339, train acc 0.854
Saved Model
Epoch: 05 | Time: 0m 33s
Epoch 5:  val loss 0.367, val acc 0.829, train loss 0.306, train acc 0.870
Saved Model
Epoch: 06 | Time: 0m 33s
Epoch 6:  val loss 0.374, val acc 0.832, train loss 0.278, train acc 0.885
Epoch: 07 | Time: 0m 32s
Epoch 7:  val loss 0.396, val acc 0.825, train loss 0.255, train acc 0.896
Epoch: 08 | Time: 0m 33s
Epoch 8:  val loss 0.423, val acc 0.810, train loss 0.233, train acc 0.906
Epoch: 09 | Time: 0m 32s
Epoch 9:  val loss 0.409, val acc 0.827, train loss 0.216, train acc 0.914
Epoch: 10 | Time: 0m 33s
Epoch 10:  val loss 0.389, val acc 0.83